# Dataset

In [1]:
#import os
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:512"

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data, Dataset
from tqdm import tqdm

E:\Studium\HPI\22-23 WS\Social Media Mining\Code\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class CustomStaticGraphTemporalSignalBatch(object):
    r"""A data iterator object to contain a static graph with a dynamically
    changing constant time difference temporal feature set (multiple signals).
    The node labels (target) are also temporal. The iterator returns a single
    constant time difference temporal snapshot for a time period (e.g. day or week).
    This single temporal snapshot is a Pytorch Geometric Batch object. Between two
    temporal snapshots the feature matrix, target matrices and optionally passed
    attributes might change. However, the underlying graph is the same.

    Args:
        edge_index (Numpy array): Index tensor of edges.
        edge_weight (Numpy array): Edge weight tensor.
        features (List of Numpy arrays): List of node feature tensors.
        targets (List of Numpy arrays): List of node label (target) tensors.
        batches (Numpy array): Batch index tensor.
        **kwargs (optional List of Numpy arrays): List of additional attributes.
    """

    def __init__(
        self,
        edge_index: Edge_Index,
        edge_weight: Edge_Weight,
        features: Node_Features,
        targets: Targets,
        batches: Batches,
        **kwargs: Additional_Features
    ):
        self.edge_index = edge_index
        self.edge_weight = edge_weight
        self.features = features
        self.targets = targets
        self.batches = batches
        self.additional_feature_keys = []
        for key, value in kwargs.items():
            setattr(self, key, value)
            self.additional_feature_keys.append(key)
        self._check_temporal_consistency()
        self._set_snapshot_count()

    def _check_temporal_consistency(self):
        assert len(self.features) == len(
            self.targets
        ), "Temporal dimension inconsistency."
        '''
        for key in self.additional_feature_keys:
            assert len(self.targets) == len(
                getattr(self, key)
            ), "Temporal dimension inconsistency."
        '''

    def _set_snapshot_count(self):
        self.snapshot_count = len(self.features)

    def _get_edge_index(self):
        if self.edge_index is None:
            return self.edge_index
        else:
            return torch.LongTensor(self.edge_index)

    def _get_batch_index(self):
        if self.batches is None:
            return self.batches
        else:
            return torch.LongTensor(self.batches)

    def _get_edge_weight(self):
        if self.edge_weight is None:
            return self.edge_weight
        else:
            return torch.FloatTensor(self.edge_weight)

    def _get_feature(self, time_index: int):
        if self.features[time_index] is None:
            return self.features[time_index]
        else:
            return torch.FloatTensor(self.features[time_index])

    def _get_target(self, time_index: int):
        if self.targets[time_index] is None:
            return self.targets[time_index]
        else:
            if self.targets[time_index].dtype.kind == "i":
                return torch.LongTensor(self.targets[time_index])
            elif self.targets[time_index].dtype.kind == "f":
                return torch.FloatTensor(self.targets[time_index])

    '''
    def _get_additional_feature(self, time_index: int, feature_key: str):
        feature = getattr(self, feature_key)[time_index]
        if feature.dtype.kind == "i":
            return torch.LongTensor(feature)
        elif feature.dtype.kind == "f":
            return torch.FloatTensor(feature)

    def _get_additional_features(self, time_index: int):
        additional_features = {
            key: self._get_additional_feature(time_index, key)
            for key in self.additional_feature_keys
        }
        return additional_features
    '''

    def __getitem__(self, time_index: Union[int, slice]):
        if isinstance(time_index, slice):
            snapshot = StaticGraphTemporalSignalBatch(
                self.edge_index,
                self.edge_weight,
                self.features[time_index],
                self.targets[time_index],
                self.batches,
                **{key: getattr(self, key)[time_index] for key in self.additional_feature_keys}
            )
        else:
            x = self._get_feature(time_index)
            edge_index = self._get_edge_index()
            edge_weight = self._get_edge_weight()
            batch = self._get_batch_index()
            y = self._get_target(time_index)
            #additional_features = self._get_additional_features(time_index)

            snapshot = Batch(x=x, edge_index=edge_index, edge_attr=edge_weight,
                            y=y, batch=batch, **additional_features)
        return snapshot

    def __next__(self):
        if self.t < len(self.features):
            snapshot = self[self.t]
            self.t = self.t + 1
            return snapshot
        else:
            self.t = 0
            raise StopIteration

    def __iter__(self):
        self.t = 0
        return self

In [ ]:
from typing import List, Union
from torch_geometric.data import Batch

Edge_Index = Union[np.ndarray, None]
Edge_Weight = Union[np.ndarray, None]
Node_Features = List[Union[np.ndarray, None]]
Targets = List[Union[np.ndarray, None]]
Batches = Union[np.ndarray, None]
Additional_Features = np.ndarray

class CustomStaticGraphTemporalSignal(object):
    r"""A data iterator object to contain a static graph with a dynamically
    changing constant time difference temporal feature set (multiple signals).
    The node labels (target) are also temporal. The iterator returns a single
    constant time difference temporal snapshot for a time period (e.g. day or week).
    This single temporal snapshot is a Pytorch Geometric Data object. Between two
    temporal snapshots the features and optionally passed attributes might change.
    However, the underlying graph is the same.

    Args:
        edge_index (Numpy array): Index tensor of edges.
        edge_weight (Numpy array): Edge weight tensor.
        features (List of Numpy arrays): List of node feature tensors.
        targets (List of Numpy arrays): List of node label (target) tensors.
        **kwargs (optional List of Numpy arrays): List of additional attributes.
    """

    def __init__(
        self,
        edge_index: Edge_Index,
        edge_weight: Edge_Weight,
        features: Node_Features,
        targets: Targets,
        **kwargs: Additional_Features
    ):
        self.edge_index = edge_index
        self.edge_weight = edge_weight
        self.features = features
        self.targets = targets
        self.additional_feature_keys = []
        for key, value in kwargs.items():
            setattr(self, key, value)
            self.additional_feature_keys.append(key)
        self._check_temporal_consistency()
        self._set_snapshot_count()

    def _check_temporal_consistency(self):
        assert len(self.features) == len(
            self.targets
        ), "Temporal dimension inconsistency."
        '''
        for key in self.additional_feature_keys:
            assert len(self.targets) == len(
                getattr(self, key)
            ), "Temporal dimension inconsistency."
        '''

    def _set_snapshot_count(self):
        self.snapshot_count = len(self.features)

    def _get_edge_index(self):
        if self.edge_index is None:
            return self.edge_index
        else:
            return torch.LongTensor(self.edge_index)

    def _get_edge_weight(self):
        if self.edge_weight is None:
            return self.edge_weight
        else:
            return torch.FloatTensor(self.edge_weight)

    def _get_features(self, time_index: int):
        return self.features[time_index]
        '''
        if self.features[time_index] is None:
            return self.features[time_index]
        else:
            return torch.FloatTensor(self.features[time_index])
        '''

    def _get_target(self, time_index: int):
        return self.targets[time_index]
        '''
        if self.targets[time_index] is None:
            return self.targets[time_index]
        else:
            if self.targets[time_index].dtype.kind == "i":
                return torch.LongTensor(self.targets[time_index])
            elif self.targets[time_index].dtype.kind == "f":
                return torch.FloatTensor(self.targets[time_index])
        '''

    '''
    def _get_additional_feature(self, time_index: int, feature_key: str):
        feature = getattr(self, feature_key)[time_index]
        if feature.dtype.kind == "i":
            return torch.LongTensor(feature)
        elif feature.dtype.kind == "f":
            return torch.FloatTensor(feature)
    
    
    def _get_additional_features(self, time_index: int):
        additional_features = {
            key: self._get_additional_feature(time_index, key)
            for key in self.additional_feature_keys
        }
        return additional_features
    '''

    def __getitem__(self, time_index: Union[int, slice]):
        if isinstance(time_index, slice):
            snapshot = CustomStaticGraphTemporalSignal(
                self.edge_index,
                self.edge_weight,
                self.features[time_index],
                self.targets[time_index],
                **{key: getattr(self, key) for key in self.additional_feature_keys}
            )
        else:
            x = self._get_features(time_index)
            edge_index = self._get_edge_index()
            edge_weight = self._get_edge_weight()
            y = self._get_target(time_index)
            #additional_features = self._get_additional_features(time_index)

            snapshot = Data(x=x, edge_index=edge_index, edge_attr=edge_weight,
                            y=y, **{key: getattr(self, key) for key in self.additional_feature_keys})
        return snapshot

    def __next__(self):
        if self.t < len(self.features):
            snapshot = self[self.t]
            self.t = self.t + 1
            return snapshot
        else:
            self.t = 0
            raise StopIteration

    def __iter__(self):
        self.t = 0
        return self

In [ ]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignalBatch

class Twibot22StaticTemporal(object):
    def __init__(self, number_of_timeframes, root=r'src/Data_test/preprocessed', device='cpu'):
        self.root = root
        self.device = device
        self.number_of_timeframes = number_of_timeframes
        path = lambda name: f"{self.root}/{name}"
        
        # load labels
        labels = torch.load(path("labels.pt"), map_location=self.device)
        labels = [labels for i in range(self.number_of_timeframes)]
    
        # load node features
        numerical_features = torch.load(path("num_properties_tensor.pt"), map_location=self.device)
        categorical_features = torch.load(path("categorical_properties_tensor.pt"), map_location=self.device)
        description_embeddings = torch.load(path("user_description_embedding_tensor.pt"), map_location=self.device)
        feature_dict = {'numerical_features': numerical_features, 'categorical_features': categorical_features, 
                       'description_embeddings': description_embeddings}
        
        tweet_embeddings = []
        for i in range(self.number_of_timeframes):
            tweet_embeddings.append(torch.load(path(f"user_tweets_tensor_{i}.pt"), map_location=self.device))
        
        
        # load edge index and types
        edge_index = torch.load(path("edge_index.pt"), map_location=self.device)
        edge_type = torch.load(path("edge_type.pt"), map_location=self.device).to(torch.float32)
    
        # load dataset masks
        train_mask = torch.load(path("train_mask.pt"), map_location=self.device)
        test_mask = torch.load(path("test_mask.pt"), map_location=self.device)
        val_mask = torch.load(path("validation_mask.pt"), map_location=self.device)
        
        self.data = CustomStaticGraphTemporalSignal(edge_index=edge_index, edge_weight=edge_type, features=tweet_embeddings, 
                                                   targets=labels, batches=self.number_of_timeframes, **feature_dict)
        
    def get_dataset():
        return self.data
    
    def get(self, idx):
        if idx == 0: return self.data
               

In [ ]:
from torch_geometric_temporal.signal import temporal_signal_split
dataset = Twibot22StaticTemporal(10, device='cpu')
train_dataset, test_dataset = temporal_signal_split(dataset.data, train_ratio=0.5)

In [ ]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 2)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [ ]:
from tqdm import tqdm
from torch_geometric_temporal.nn.recurrent import GConvGRU

model = RecurrentGCN(node_features=14, filters=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(50)):
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

In [2]:
class Twibot22(Dataset):
    def __init__(self, root=r'src/Data_test/preprocessed', device='cpu'):
        self.root = root
        super().__init__(self.root, None, None, None)
        self.device = device
        path = lambda name: f"{self.root}/{name}"
        
        # load labels
        labels = torch.load(path("labels.pt"), map_location=self.device)
        
        # load node features
        numerical_features = torch.load(path("num_properties_tensor.pt"), map_location=self.device)
        categorical_features = torch.load(path("categorical_properties_tensor.pt"), map_location=self.device)
        description_embeddings = torch.load(path("user_description_embedding_tensor.pt"), map_location=self.device)
        tweet_embeddings = torch.load(path("user_tweets_tensor.pt"), map_location=self.device)
        #merged_features = torch.cat([numerical_features, categorical_features, description_embeddings, tweet_embeddings], dim=1)
        
        # load edge index and types
        edge_index = torch.load(path("edge_index.pt"), map_location=self.device)
        edge_type = torch.load(path("edge_type.pt"), map_location=self.device)
        
        # load dataset masks
        train_mask = torch.load(path("train_mask.pt"), map_location=self.device)
        test_mask = torch.load(path("test_mask.pt"), map_location=self.device)
        val_mask = torch.load(path("validation_mask.pt"), map_location=self.device)
        
        
        '''
        # create data object
        self.data = Data(x=merged_features, edge_index=edge_index, edge_attr=edge_type, y=labels).to(self.device)
        self.data.train_mask = train_mask
        self.data.test_mask = test_mask
        self.data.val_mask = val_mask
        self.data.description_embeddings = description_embeddings
        self.data.tweet_embeddings = tweet_embeddings
        self.data.numerical_features = numerical_features
        self.data.categorical_features = categorical_features
        '''
        
        self.data = Data(
            edge_index=edge_index,
            edge_attr=edge_type,
            y=labels,
            description_embeddings = description_embeddings,
            tweet_embeddings = tweet_embeddings,
            numerical_features = numerical_features,
            categorical_features = categorical_features,
            train_mask = train_mask,
            test_mask = test_mask,
            val_mask = val_mask,
            num_nodes = labels.shape[0]
        )
        
        assert self.data.validate()
        
    def len(self):
        return 1
    
    def get(self, idx):
        if idx == 0: return self.data
    
    @property
    def num_node_features(self):
        return self.data.num_node_features
    
    @property
    def num_edge_features(self):
        return self.data.num_edge_features
    
    @property
    def num_nodes(self):
        return self.data.num_nodes
    
    @property
    def num_edges(self):
        return self.data.num_edges  

# Model

In [3]:
import torch
from torch import nn
from torch_geometric import nn as gnn

In [4]:
class BotRGCN(nn.Module):
    def __init__(self, desc_embedding_size=768, tweet_embedding_size=768, num_feature_size=5, 
                 cat_feature_size=3, embedding_dimension=128, num_relations=2, dropout=0.3):
        super().__init__()
        self.dropout = dropout
        
        # TODO: use torch_geometric.nn.Sequential instead?
        
        # user description layer
        self.desc_layer = nn.Sequential(
            nn.Linear(desc_embedding_size, int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        
        # user tweet layer
        self.tweet_layer = nn.Sequential(
            nn.Linear(tweet_embedding_size, int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        
        # numeric feature layer
        self.num_feature_layer = nn.Sequential(
            nn.Linear(num_feature_size, int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        
        # categorical feature layer
        self.cat_feature_layer = nn.Sequential(
            nn.Linear(cat_feature_size, int(embedding_dimension/4)),
            nn.LeakyReLU()  
        )
        
        self.inner = gnn.Sequential('x, edge_index, edge_type', [
            (nn.Linear(embedding_dimension, embedding_dimension), 'x -> x'),
            (nn.LeakyReLU(), 'x -> x'),
            (gnn.RGCNConv(embedding_dimension, embedding_dimension, num_relations=num_relations), 'x, edge_index, edge_type -> x'),
            (nn.Dropout(self.dropout), 'x -> x'),
            (gnn.RGCNConv(embedding_dimension, embedding_dimension, num_relations=num_relations), 'x, edge_index, edge_type -> x'),
            (nn.Linear(embedding_dimension, embedding_dimension), 'x -> x'),
            (nn.LeakyReLU(), 'x -> x'),
            (nn.Linear(embedding_dimension, 1), 'x -> x'),
            (nn.Sigmoid(), 'x -> x'),
            ])
        
        '''
        # embedding layer
        self.embedding_input_layer = nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()  
        )
        
        # RGCN layer
        # TODO: replace with FastRGCNConv?
        self.rgcn_layer = gnn.RGCNConv(embedding_dimension, embedding_dimension, num_relations=num_relations)
        
        # embedding output layer
        self.embedding_output_layer_1 = nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()  
        )
        
        # output layer
        self.output_layer = nn.Linear(embedding_dimension, 2)
        '''
        
    def forward(self, desc_embedding, tweet_embedding, num_feature, cat_feature, edge_index, edge_type):        
        desc = self.desc_layer(desc_embedding)
        tweets = self.tweet_layer(tweet_embedding)
        numeric = self.num_feature_layer(num_feature)
        cat = self.cat_feature_layer(cat_feature)
        x = torch.cat([desc, tweets, numeric, cat], dim=1)
        
        return self.inner(x, edge_index, edge_type).flatten()
    
    def init_weights(self, m):
        if type(m) == nn.Linear:
            nn.init.kaiming_uniform_(m.weight)

# Training

In [5]:
torch.set_printoptions(threshold=100)

In [6]:
'''
from torchmetrics import MetricCollection
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
from torch_geometric.loader import NeighborLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
embedding_size = 128
dropout = 0.3
lr = 1e-3
weight_decay = 5e-3

dataset = Twibot22(device=device)
model = BotRGCN(desc_embedding_size=768, tweet_embedding_size=768, num_feature_size=5, 
                 cat_feature_size=3, embedding_dimension=128, num_relations=2, dropout=0.3)
model.apply(model.init_weights)

model_metrics = MetricCollection([
    BinaryAccuracy(), 
    BinaryPrecision(), 
    BinaryRecall(), 
    BinaryF1Score()])

model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(
        dataset.description_embeddings,
        dataset.tweet_embeddings, 
        dataset.numerical_features,
        dataset.categorical_features.to(torch.float32),
        dataset.edge_index,
        dataset.edge_type)
    #print(out[dataset.train_mask][0:50])
    #print(dataset.labels[dataset.train_mask][0:50])
    loss = criterion(out[dataset.train_mask], dataset.labels[dataset.train_mask].to(torch.float32))
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out = model(
        dataset.description_embeddings,
        dataset.tweet_embeddings, 
        dataset.numerical_features,
        dataset.categorical_features.to(torch.float32),
        dataset.edge_index,
        dataset.edge_type)
    #preds = torch.argmax(out[dataset.test_mask], dim=1).to('cpu').detach()
    preds = out[dataset.test_mask].to('cpu').detach()
    labels = dataset.labels[dataset.test_mask].to('cpu').detach()
    metrics = model_metrics(preds, labels)
    return metrics
    
losses = []
for idx, e in tqdm(enumerate(range(0, 20000))):
    loss = train()
    losses.append(loss.item())
metrics = test()
print(metrics)
''';

In [7]:
import os
from collections import defaultdict
from datetime import datetime

import pandas as pd

from torchmetrics import MetricCollection
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score, BinaryMatthewsCorrCoef
from torch_geometric.loader import NeighborLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
embedding_size = 128
dropout = 0.3
lr = 1e-3
weight_decay = 5e-3
batch_size=1024
training_epochs = 10
num_neighbors = [256] * 4

dataset = Twibot22(device='cpu')
model = BotRGCN(desc_embedding_size=768, tweet_embedding_size=768, num_feature_size=5, 
                 cat_feature_size=3, embedding_dimension=128, num_relations=2, dropout=0.3)
model.apply(model.init_weights)
model.to(device)

model_metrics = MetricCollection([
    BinaryAccuracy(), 
    BinaryPrecision(), 
    BinaryRecall(), 
    BinaryF1Score(), 
    BinaryMatthewsCorrCoef()])

model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

data = dataset[0]
train_loader = NeighborLoader(data, num_neighbors=num_neighbors, batch_size=batch_size, input_nodes=data.train_mask, shuffle=True)
test_loader = NeighborLoader(data, num_neighbors=num_neighbors, batch_size=batch_size, input_nodes=data.test_mask)
validation_loader = NeighborLoader(data, num_neighbors=num_neighbors, batch_size=batch_size, input_nodes=data.val_mask)


def train_epoch():
    model.train()
    average_loss = 0.0
    
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(
            batch.description_embeddings,
            batch.tweet_embeddings, 
            batch.numerical_features,
            batch.categorical_features.to(torch.float32),
            batch.edge_index,
            batch.edge_attr)
        loss = criterion(out[:batch_size], batch.y[:batch_size].to(torch.float32))
        average_loss += loss.item() * batch_size
        loss.backward()
        optimizer.step()
    
    average_loss /= len(train_loader) * batch_size
        
    return average_loss

def evaluate():
    model.eval()
    
    labels = []
    predictions = []
    
    with torch.no_grad():
        for batch in test_loader:
            batch.to(device)
            out = model(
                batch.description_embeddings,
                batch.tweet_embeddings, 
                batch.numerical_features,
                batch.categorical_features.to(torch.float32),
                batch.edge_index,
                batch.edge_attr)
            
            labels.extend(list(batch.y[:batch_size].to('cpu')))
            predictions.extend(list(out[:batch_size].to('cpu')))
        metrics = model_metrics(torch.tensor(predictions), torch.tensor(labels))
    return metrics

losses = []
test_metrics = defaultdict(list)

for e in range(training_epochs+1):
    loss = train_epoch()
    losses.append(loss)
    
    if e % 10 == 0:
        metrics = evaluate()
        test_metrics['epoch'].append(e)
        for k, v in metrics.items():
            test_metrics[k].append(v.item())
        
        print(f"Epoch: {e}, Loss: {loss:.2f}")
        print(f"Accuracy: {metrics['BinaryAccuracy']:.2f}")
        print(f"Precision: {metrics['BinaryPrecision']:.2f}")
        print(f"Recall: {metrics['BinaryRecall']:.2f}")
        print(f"F1-Score: {metrics['BinaryF1Score']:.2f}")
        print(f"MCC: {metrics['BinaryMatthewsCorrCoef']:.2f}")
        print()

        
# save loss, metrics and state dict
test_metrics_df = pd.DataFrame(test_metrics)     
losses_df = pd.DataFrame(losses, columns=['BCE loss'])

time = datetime.now().strftime("%d_%m_%Y__%H_%M_%S")
os.makedirs('results', exist_ok=True)

test_metrics_df.to_csv(f'results/test_metrics_{time}.csv')
losses_df.to_csv(f'results/losses_{time}.csv')
torch.save(model.state_dict(), f'results/state_dict_{time}.pt')

Epoch: 0, Loss: 0.24
Accuracy: 0.80
Precision: 0.53
Recall: 0.03
F1-Score: 0.05
MCC: 0.08

Epoch: 10, Loss: 0.12
Accuracy: 0.81
Precision: 0.54
Recall: 0.14
F1-Score: 0.22
MCC: 0.20



In [12]:
torch.cuda.empty_cache()

In [9]:
os.makedirs("bla/", exist_ok=True)

In [8]:
metrics_df = pd.read_csv('test_metrics_11_01_2023__18_32_00.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'test_metrics_11_01_2023__18_32_00.csv'

In [ ]:
m2 = torch.load("results/state_dict_11_01_2023__18_37_43.pt")

In [ ]:
losses_df = pd.DataFrame(losses, columns=['BCE loss'])

In [ ]:
losses_df

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)

In [ ]:
data.train_mask.device

In [ ]:
data.validate()

In [ ]:
#next(iter(train_loader))

In [ ]:
len(train_loader)* batch_size

In [33]:
torch.cuda.empty_cache()